In [3]:
import os
from dotenv import load_dotenv
from langchain_core.tools import tool
from x_sem_ad import XSemAD, load_event_log_from_xes
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage

load_dotenv()

if not os.getenv("GOOGLE_API_KEY"):
    raise ValueError("A chave de API do Google não foi encontrada. Defina a variável de ambiente GOOGLE_API_KEY.")

In [4]:
path_to_model="./data/model/"
path_to_log = 'data/InternationalDeclarations.xes'

print("Inicializando o sistema de análise de anomalias...")
log = load_event_log_from_xes(path_to_log)
model = XSemAD(path_to_model=path_to_model)
model.load_event_log(log)

Inicializando o sistema de análise de anomalias...


/home/danilo/dev/research/venv/lib/python3.10/site-packages/pm4py/util/dt_parsing/parser.py:82: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(


parsing log, completed traces ::   0%|          | 0/6449 [00:00<?, ?it/s]

Loading model from: ./data/model/ folder


In [5]:
@tool
def analisar_anomalias_de_processo(constraint_type: str,
								   threshold: float=0.75) -> dict:
	"""
	Executa a análise de anomalias para um tipo de restrição específico em um log de eventos de processo já carregado.

	Args:
		constraint_type: O tipo de restrição a ser verificada. A partir da pergunta do usuário, escolha um dos seguintes valores: 'Init', 'End', 'Precedence', 'Succession', 'Alternate Precedence', 'Co-Existence', 'Response', 'Alternate Response', 'Alternate Succession', 'Choice', 'Exclusive Choice'. Para "regra de início", use 'Init'.
		threshold: O valor de limiar para a análise de anomalias. Este parâmetro é usado para definir o nível de severidade das violações de restrição.

	Returns:
		Um dicionário com as restrições violadas e a contagem de suas violações,
		no seguinte formato: {
			"constraint_type[activity_a, activity_b]": int(violation_count), ...
		}, exemplo:
		{
			"Init[Activity A, Activity B]": 5,
			"End[Activity C, Activity D]": 3
	"""
	print(f"\n>>> analisar_anomalias_de_processo(constraint_type='{constraint_type}', threshold={threshold})")
	result = model.run(constraint_type=constraint_type, threshold=threshold)
	print(f">>> {result}\n")
	return result

In [ ]:
MODEL = "gemini-2.0-flash"
# MODEL = "gemini-2.5-flash"
# MODEL = "gemini-2.5-pro"

llm = ChatGoogleGenerativeAI(model=MODEL, temperature=0)
tools = [analisar_anomalias_de_processo]
llm_with_tools = llm.bind_tools(tools)

with open("./data/qa_prompt.txt", "r") as file:
	system_prompt = file.read()
	print(system_prompt)

messages = [SystemMessage(content=system_prompt)]

You are an expert Process Mining assistant. Your primary goal is to help users analyze and understand event logs by using the tools at your disposal.

Follow these rules strictly:
1. Translate the constraints rules and the activities to the language of the user.
2. Answer in natural language, so that any user not familiar with the process mining can understand your answer without any technical knowledge.
3. If context is provided, use it to enhance your response and judge the correctness of the tool responses.
4. If no anomalies are found, state this clearly and positively.
5. End with a brief summary or suggestion, if applicable.

Here are some examples of how you should respond based on the conversation history:

**EXAMPLE 1: ANOMALIES FOUND**

*Question:*
Quais são as anomalias para a regra de início e coexistência?

*Tool Responses:*
{'Start[register request]': 5}
{'Co-Existence[approve order, ship goods]': 12}

*Correct Final Answer:*
Olá! Analisei o log de eventos e encontrei as 

In [21]:
with open("./data/context.txt", "r") as file:
	system_prompt = file.read()

messages.append(SystemMessage(content=system_prompt))

In [22]:
user_prompt = "Quais são as anomalias que quebram a regra de fim no log de eventos carregado? Considere um limiar de 80% para a análise."
messages.append(HumanMessage(content=user_prompt))

ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

print(f"🧑‍💻 Pergunta: {user_prompt}")
if ai_msg.tool_calls:
	for tool_call in ai_msg.tool_calls:
		selected_tool = {t.name: t for t in tools}[tool_call["name"]]
		tool_output = selected_tool.invoke(tool_call["args"])

		tool_message = ToolMessage(content=str(tool_output),
									tool_call_id=tool_call['id'])
		messages.append(tool_message)
	final_response = llm_with_tools.invoke(messages)

	print(f"‍💻 Resposta: {final_response.content}")
else:
	print(f"‍💻 Resposta: {ai_msg.content}")


🧑‍💻 Pergunta: Quais são as anomalias que quebram a regra de fim no log de eventos carregado? Considere um limiar de 80% para a análise.

>>> analisar_anomalias_de_processo(constraint_type='End', threshold=0.8)
Get prediction for constraint type:  End
>>> {'End[declaration rejected by supervisor]': 6448}

‍💻 Resposta: Olá! Analisei o log de eventos para você.

Com um limiar de 80%, encontrei violações para a regra de *Fim*. A análise mostrou que a atividade *declaração rejeitada pelo supervisor* apareceu como a atividade final em **6.448** casos.

Isso indica que um número significativo de processos de despesas de viagem não termina com o pagamento ou a conclusão da viagem, como esperado, mas sim com a rejeição da declaração pelo supervisor. Esta é uma anomalia importante, pois representa um desfecho negativo e frequente no processo.
